In [41]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# إعداد Augmentation + تقسيم البيانات
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% للـ validation
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# تحميل البيانات وتوحيد الحجم
train_data = datagen.flow_from_directory(
    '../data/1000 QR Images/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    '../data/1000 QR Images/val',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# تحميل MobileNetV2 بدون الطبقات العلوية
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False  # تثبيت الطبقات أثناء أول تدريب

# إضافة الطبقات العليا
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# تجميع النموذج
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# حفظ أفضل نموذج
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy", mode="max")
early = EarlyStopping(patience=5, restore_best_weights=True)

# تدريب النموذج
model.fit(train_data, validation_data=val_data, epochs=20, callbacks=[checkpoint, early])

# حفظ النموذج النهائي
model.save("qr_spoofing_detector.h5")



Found 654 images belonging to 2 classes.
Found 34 images belonging to 2 classes.


C:\Users\Eman\AppData\Local\Temp\ipykernel_13288\1503087569.py:38: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 813ms/step - accuracy: 0.5443 - loss: 0.6921

21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 972ms/step - accuracy: 0.5449 - loss: 0.6917 - val_accuracy: 0.7059 - val_loss: 0.6009
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 863ms/step - accuracy: 0.6070 - loss: 0.6542

21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 961ms/step - accuracy: 0.6081 - loss: 0.6534 - val_accuracy: 0.7941 - val_loss: 0.5595
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 896ms/step - accuracy: 0.6634 - loss: 0.6115 - val_accuracy: 0.7941 - val_loss: 0.5615
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 947ms/step - accuracy: 0.7286 - loss: 0.5824 - val_accuracy: 0.7941 - val_loss: 0.5138
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 922ms/step - accuracy: 0.7332 - loss: 0.5655 - val_accuracy: 0.7941 - val_loss: 0.4753
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 18s 864ms/step - accuracy: 0.6969 - loss: 0.6014 - val_accuracy: 0.7941 - val_loss: 0.4683
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 932ms/step - accuracy: 0.7388 - loss: 0.5635 - val_accuracy: 0.7353 - val_loss: 0.5164
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 895ms/step - accuracy: 0.7267 - loss: 0.5888 - val_accuracy: 0.7647 - val_loss: 0.5461
Epoch 9/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 810ms/step - accuracy: 0.7513 - loss: 0.5472

21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 905ms/step - accuracy: 0.7517 - loss: 0.5476 - val_accuracy: 0.8235 - val_loss: 0.4674
Epoch 10/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 883ms/step - accuracy: 0.7536 - loss: 0.5668 - val_accuracy: 0.7647 - val_loss: 0.5008
Epoch 11/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 891ms/step - accuracy: 0.7402 - loss: 0.5593 - val_accuracy: 0.8235 - val_loss: 0.5233
Epoch 12/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 897ms/step - accuracy: 0.7594 - loss: 0.5624 - val_accuracy: 0.8235 - val_loss: 0.4612
Epoch 13/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 896ms/step - accuracy: 0.7856 - loss: 0.5475 - val_accuracy: 0.8235 - val_loss: 0.4537
Epoch 14/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 897ms/step - accuracy: 0.7534 - loss: 0.5520 - val_accuracy: 0.7941 - val_loss: 0.5090
Epoch 15/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 923ms/step - accuracy: 0.7690 - loss: 0.5411 - val_accuracy: 0.7941 - val_loss: 0.4681
Epoch 16/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.7642 - loss: 0.5356 - val_accuracy: 0

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img('../data/1000 QR Images/test/benign_qr_images_500/qr_497_benign_images.png', target_size=(256, 256))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
print("Spoofing" if prediction[0][0] > 0.5 else "Legit")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Spoofing


using YOLO

In [1]:
# Predict using a pretrained YOLO model (e.g., YOLO11n) on an image
!yolo predict model=yolo11m.pt source='https://ultralytics.com/images/bus.jpg'

Ultralytics 8.3.115  Python-3.13.2 torch-2.7.0+cpu CPU (Intel Core(TM) i7-8550U 1.80GHz)
YOLO11m summary (fused): 125 layers, 20,091,712 parameters, 0 gradients, 68.0 GFLOPs

image 1/1 d:\Eman Folder\Training\ML\CLS Learning Solutio-Various Files-Microsoft Machine Learning new-20241028-108a\project\16-Apr 2025\TrustSentinel-main\models\sentiment\notebooks\bus.jpg: 640x480 4 persons, 1 bus, 618.5ms
Speed: 7.2ms preprocess, 618.5ms inference, 14.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict
 Learn more at https://docs.ultralytics.com/modes/predict



  0%|          | 0.00/38.8M [00:00<?, ?B/s]
  0%|          | 128k/38.8M [00:00<00:31, 1.27MB/s]
  1%|▏         | 512k/38.8M [00:00<00:14, 2.74MB/s]
  3%|▎         | 1.00M/38.8M [00:00<00:10, 3.70MB/s]
  4%|▍         | 1.50M/38.8M [00:00<00:09, 4.15MB/s]
  5%|▌         | 2.00M/38.8M [00:00<00:08, 4.39MB/s]
  6%|▋         | 2.50M/38.8M [00:00<00:08, 4.52MB/s]
  8%|▊         | 3.00M/38.8M [00:00<00:08, 4.61MB/s]
  9%|▉         | 3.50M/38.8M [00:00<00:07, 4.66MB/s]
 10%|█         | 4.00M/38.8M [00:00<00:07, 4.71MB/s]
 12%|█▏        | 4.50M/38.8M [00:01<00:07, 4.73MB/s]
 13%|█▎        | 5.00M/38.8M [00:01<00:07, 4.75MB/s]
 14%|█▍        | 5.50M/38.8M [00:01<00:07, 4.76MB/s]
 15%|█▌        | 6.00M/38.8M [00:01<00:07, 4.77MB/s]
 17%|█▋        | 6.50M/38.8M [00:01<00:07, 4.77MB/s]
 18%|█▊        | 7.00M/38.8M [00:01<00:06, 4.78MB/s]
 19%|█▉        | 7.50M/38.8M [00:01<00:06, 4.78MB/s]
 21%|██        | 8.00M/38.8M [00:01<00:06, 4.79MB/s]
 22%|██▏       | 8.50M/38.8M [00:01<00:06, 4.79MB/s]
 23

In [6]:
import os
import shutil
import random
from pathlib import Path
import yaml

# تعديل المسار حسب مكان مجلد الصور
ORIGINAL_FOLDER = "../data/1000 QR Images"
NEW_FOLDER = "qr_dataset"

# إنشاء المجلدات الجديدة
for split in ['train', 'val', 'test']:
    os.makedirs(f"{NEW_FOLDER}/images/{split}", exist_ok=True)
    os.makedirs(f"{NEW_FOLDER}/labels/{split}", exist_ok=True)

# قراءة كل الصور
all_images = [f for f in os.listdir(ORIGINAL_FOLDER) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(all_images)

# تقسيم البيانات
total = len(all_images)
train_split = int(total * 0.7)
val_split = int(total * 0.9)

splits = {
    'train': all_images[:train_split],
    'val': all_images[train_split:val_split],
    'test': all_images[val_split:]
}

# وظيفة لكتابة ملف label مؤقت
def write_label(filename, label, split):
    label_path = os.path.join(NEW_FOLDER, "labels", split, Path(filename).stem + ".txt")
    with open(label_path, "w") as f:
        f.write(f"{label} 0.5 0.5 1 1\n")  # تنسيق YOLO: class cx cy w h (full image)

# نقل الصور وكتابة التصنيفات
for split, images in splits.items():
    for img in images:
        label = 1 if 'mal' in img.lower() else 0
        src = os.path.join(ORIGINAL_FOLDER, img)
        dst = os.path.join(NEW_FOLDER, "images", split, img)
        shutil.copy2(src, dst)
        write_label(img, label, split)

# إنشاء ملف data.yaml
data_yaml = {
    'train': f"{NEW_FOLDER}/images/train",
    'val': f"{NEW_FOLDER}/images/val",
    'test': f"{NEW_FOLDER}/images/test",
    'nc': 2,
    'names': ['legit', 'malicious']
}

with open(f"{NEW_FOLDER}/data.yaml", "w") as f:
    yaml.dump(data_yaml, f)

print("✅ Dataset ready for YOLO training!")


✅ Dataset ready for YOLO training!


In [1]:
from ultralytics import YOLO

model = YOLO("yolo11l.pt")  # أو yolov8s.yaml لو الجهاز قوي
model.train(data="qr_dataset/data.yaml", epochs=20, imgsz=256)

Ultralytics 8.3.115  Python-3.13.2 torch-2.7.0+cpu CPU (Intel Core(TM) i7-8550U 1.80GHz)
engine\trainer: task=detect, mode=train, model=yolo11l.pt, data=qr_dataset/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width

train: Scanning D:\Eman Folder\Training\ML\CLS Learning Solutio-Various Files-Microsoft Machine Learning new-20241028-108a\project\16-Apr 2025\TrustSentinel-main\models\sentiment\notebooks\qr_dataset\labels\train.cache... 0 images, 816 backgrounds, 0 corrupt: 100%|██████████| 816/816 [00:00<?, ?it/s]

WARNING No labels found in D:\Eman Folder\Training\ML\CLS Learning Solutio-Various Files-Microsoft Machine Learning new-20241028-108a\project\16-Apr 2025\TrustSentinel-main\models\sentiment\notebooks\qr_dataset\labels\train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: Fast image access  (ping: 0.30.0 ms, read: 0.00.0 MB/s, size: 0.6 KB)



c:\Users\Eman\anaconda3\envs\condaCV\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning D:\Eman Folder\Training\ML\CLS Learning Solutio-Various Files-Microsoft Machine Learning new-20241028-108a\project\16-Apr 2025\TrustSentinel-main\models\sentiment\notebooks\qr_dataset\labels\val.cache... 0 images, 176 backgrounds, 0 corrupt: 100%|██████████| 176/176 [00:00<?, ?it/s]

WARNING No labels found in D:\Eman Folder\Training\ML\CLS Learning Solutio-Various Files-Microsoft Machine Learning new-20241028-108a\project\16-Apr 2025\TrustSentinel-main\models\sentiment\notebooks\qr_dataset\labels\val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

                   all        176          0          0          0          0          0
WARNING no labels found in detect set, can not compute metrics without labels



c:\Users\Eman\anaconda3\envs\condaCV\Lib\site-packages\ultralytics\utils\metrics.py:655: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
c:\Users\Eman\anaconda3\envs\condaCV\Lib\site-packages\numpy\_core\_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/20         0G          0     0.1989          0          0        256:  76%|███████▋  | 39/51 [15:25<04:44, 23.74s/it]


KeyboardInterrupt: 